<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Datenanalyse und Datenmanagement</h1></div>
</div>

---
# Notebook II: Data Management
In diesem Notebook geht es um die Arbeit mit Datenbanken und APIs.

## Inhaltsverzeichnis

[1. Einführung in SQL mit Notebooks](#kapitel1)  
[2. Erstellen von Tabellen und Daten](#kapitel2)  
[3. Abfragen von Daten](#kapitel3)  
[4. Verbindung zu Datenbankservern herstellen](#kapitel4)  
[5. Verbindung zu APIs herstellen](#kapitel5)  
[6. Übungen Datenbank- und API-Abfragen](#kapitel6)  

---

## 1. Einführung in SQL mit Notebooks <a id="kapitel1"/>

Obwohl wir mit einem Python3-Notebook, können wir in einer Zelle temporär andere Sprachen verwenden. Hierzu müssen wir eine Notebook-Extension laden (in unserem Beispiel ```sql```).

In [2]:
%load_ext sql

Unsere ersten Schritte machen wir mit einer temporär erzeugten Datenbank vom Typ SQLite (mehr Infos dazu [hier](https://www.sqlite.org/index.html)). SQLite Datenbanken liegen typischerweise als eine einzelne Datei vor und die SQLite Software kann flexibel in Programme in verschiedensten Sprachen eingebunden werden.

Deren Website beschreibt SQLite so:  
> SQLite [...] implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day.

Die Dokumentation zu den unterstützen SQL-Befehlen in SQLite findest du [hier](https://sqlite.org/lang.html).

Erzeugen wir nun unsere Datenbank, indem wir mit dem Magic ```%%sql``` Jupyter mitteilen, dass dies eine SQL-Zelle ist und nachfolgend eine SQLite-Verbindung definieren:

In [ ]:
%%sql 

sqlite://

## 2. Erstellen von Tabellen und Daten <a id="kapitel2"/>
SQL-Befehle sind etwas anders aufgebaut als Python-Befehle. Für unsere Zwecke beachte zwei Dinge:
- Ein Befehl ist mit ```;``` abzuschließen, sonst wird er nicht ausgeführt
- Kommentare sind mit ```--```einzuleiten statt mit ```#``` 

In den Kapitel 2 und 3 werden wir eine Datenstruktur erzeugen, wie sie in folgendem Klassendiagramm abgebildet ist:  
<img style="width:450px;" src="images/02_class-diagram.png"/>

### 2.1. Erzeugen von Tabellen
Erzeugen wir nun im ersten Schritt mit SQL die Tabelle "Mitarbeiter" mit dem Befehl ```CREATE TABLE```.  

Auf deinem Smartphone würden diese Kommandos beispielsweise beim ersten Start durch eine App ausgeführt.

In [ ]:
%%sql  

-- Falls es die Tabelle schon geben sollte (etwa weil du die Zelle schon einmal ausgeführt hast): Tabelle löschen
DROP TABLE IF EXISTS Mitarbeiter; 

CREATE TABLE Mitarbeiter (          -- Erzeuge die Tabelle Mitarbeiter
    mitarbeiter_nr int PRIMARY KEY, -- ... mit einer Spalte "nr" vom Typ integer (Ganzzahl), die der Primärschlüssel ist
    vorname varchar(50),            -- ... und den Spalten Vorname und Name, die vom Typ Varchar (Zeichenkette) der maximalen Länge 50 sind
    name varchar(50));  

Nun fügen wir Datensätze hinzu:

In [ ]:
%%sql

INSERT INTO Mitarbeiter VALUES (1, "Marco", "Lind");
INSERT INTO Mitarbeiter VALUES (2, "Peter", "Lind");
INSERT INTO Mitarbeiter VALUES (3, "Sophia", "Lind");

---
### <span style="color:#FF5D02;">Aufgabe 1: Datenstruktur erweitern</span>
1. Erzeuge die zweite Tabelle "Abteilungen" wie oben im Klassendiagramm beschrieben
2. Füge drei Abteilungen hinzu: Einkauf, Produktion, Vertrieb

In [ ]:
%%sql



In [ ]:
%%sql



---

### 2.2. Tabellen verknüpfen

Je nach Art der Assoziation erfolgt die Umsetzung in der Datenbank verschieden:
- Typ **1..1 (auch: 1:1)**: Die Attribute beider Tabellen werden meist in eine Tabelle zusammengefasst.

- Typ **1..\* (auch: 1:n)**: In der Tabelle auf der "n-Seite" wird eine Fremdschlüssel-Spalte zum Verweis auf die zweite Tabelle erzeugt  
_Beispiel: Die Mitarbeiter-Tabelle erhält eine Spalte "abteilungs_nr", die für jeden Mitarbeiter per Id auf eine Abteilung verweist_


- Typ **\*..\* (auch n:m)**: In einer Verknüpfungstabelle wird mit zwei Fremdschlüssel auf beide Tabellen verwiesen.  
_Beispiel: Könnten Mitarbeiter in mehreren Abteilungen sein, würde eine Tabelle "Abteilungszugehörigkeit" diese im Format (mitarbeiter_id, abteilungs_id) speichern_



Verknüpfen wir nun unsere Tabellen wie im Klassendiagramm dargestellt.  

<img style="width:450px;float:left;" src="images/02_class-diagram.png"/>

Gemäß obiger Logik müssen wir der Mitarbeiter-Tabelle eine Spalte "abteilungs_nr" als Fremdschlüssel hinzufügen.  
Der Einfachheit halber erzeugen wir die Tabelle neu und legen unsere Beispielmitarbeiter ergänzt um Verweise auf die Abteilungen an.

In [ ]:
%%sql 

DROP TABLE IF EXISTS Mitarbeiter; 

CREATE TABLE Mitarbeiter (
    mitarbeiter_nr int PRIMARY KEY,
    abteilungs_nr int,                    -- hier haben wir die Id der Abteilung als Fremdschlüssel eingefügt
    vorname varchar(50),
    name varchar(50),
    FOREIGN KEY (abteilungs_nr) REFERENCES Abteilungen(abteilungs_nr)
    );  

INSERT INTO Mitarbeiter VALUES (1, 2, "Marco", "Lind");  -- id "1", Abteilungsnr. "2", Vorname "Marco", Nachname "Lind"
INSERT INTO Mitarbeiter VALUES (2, 2, "Peter", "Lind");
INSERT INTO Mitarbeiter VALUES (3, 3, "Sophia", "Lind");

### Wozu das Ganze?
Wenn wir nun das Überprüfen der Fremdschlüssel (foreign keys) scharf schalten, kann die Datenbank die Datenkonsistenz sicherstellen:  
- **Referenzielle Integrität**: Wir können keine Mitarbeiter mit ungültigen Abteilungs-Ids anlegen und keine Abteilungen löschen, denen noch Mitarbeiter zugeordnet sind.
- **Vermeiden von Redundanz**: Müssten wir zu jedem Mitarbeiter den Abteilungsnamen ausschreiben, wäre das Umbenennen von Abteilungen aufwändig - jeder Mitarbeiterdatensatz müsste geändert werden. 

Auch vermeiden wir so inkonsistente Schreibweisen der Abteilungsnamens und andere Fehlerquellen.  
Probieren wir es aus:

In [ ]:
%%sql 
PRAGMA foreign_keys=1;                                          -- Überprüfung der Fremdschlüssel "scharf" schalten
INSERT INTO Mitarbeiter VALUES (7, 25, "Gerhard", "Lind");      -- schlägt fehl: Abteilungs_nr 25 gibt es nicht

## 3. Abfragen von Daten <a id="kapitel3"/>

### 3.1. Daten aus einzelnen Tabellen
Nun ist es naheliegend, diese Daten wieder Abfragen zu wollen. Dies geschieht mit dem ```SELECT``` Befehl. Eine Smartphone-App würde mittels SELECT  beispielsweise beim Start die benötigten Daten zur Anzeige aus dem Handyspeicher laden.

In [ ]:
%%sql

SELECT vorname, name FROM Mitarbeiter;

Anstelle einer Auflistung der Datenfelder können mit ```*``` alle Felder abgefragt werden.  
Mit dem Unterbefehl ```WHERE``` kannst du die Ergebnisdaten filtern, mit ```ORDER BY``` sortieren:

In [ ]:
%%sql

SELECT * FROM Mitarbeiter
WHERE abteilungs_nr = 2           -- nur aus Abteilung
ORDER BY vorname;                 -- alphabetisch nach Vorname sortieren

---
### <span style="color:#FF5D02;">Aufgabe 2: Überprüfe die Tabelle "Abteilungen"</span>
Überprüfe, ob deine Tabelle "Abteilungen" korrekt erzeugt wurde, indem du die Daten mittels ```SELECT``` abfragst


In [ ]:
%%sql



Du solltest nun eine Tabelle erzeugt haben, die so aussieht:
abteilungs_nr | bezeichnung
---|:---
1 | Einkauf
2 | Produktion
3 | Vertrieb

Überprüfe dein Ergebnis!

---

### 3.2. Abfrage aus verknüpften Tabellen
Nun wollen wir aber die Abteilungsnamen nicht manuell nachschlagen, sondern beide Tabelle in einer Abfrage zusammenführen.


In [ ]:
%%sql

SELECT mitarbeiter_nr, vorname, name, bezeichnung AS Abteilung            -- wir benennen hier die Spalte "bezeichnung" in "Abteilung" um
FROM Mitarbeiter, Abteilungen
WHERE Mitarbeiter.abteilungs_nr = Abteilungen.abteilungs_nr;              -- Die Abteilungsnr. aus Tabelle Mitarbeiter und Abteilung muss übereinstimmen

## 4. Daten von Datenbankservern abrufen <a id="kapitel4"/>

Nach diesen Trockenübungen mit einer temporären SQLite-Datenbank in einer Datei stellen wir nun die Verbindung zu einer vorbereiteten Datenbank her. Hierzu geben wir eine URL an, ähnlich den Website-Adressen in der Browserzeile. Der Aufbau ist im Allgemeinen wie folgend: 
> _Datenbanktyp_://_Username_:_Password_@_Servername_/_Datenbankname_ 

Für SQlite-Datenbanken im Dateisystem des Notebooks:

> _sqlite:///**pfad/zur/datenbankdatei**_

Wir stellen nun eine Verbindung zu einer SQLite-Datenbank in der Datei ```smi-data.db``` im Unterverzeichnis ```data``` her.

In [14]:
%sql sqlite:///data/smi-data.db

SQLite hebt datenbankinterne Informationen in einer automatisch angelegten Tabelle ```sqlite_master```an.  
Hier lässt sich beispielsweise abfragen, welche Tabellen es gibt.

In [17]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///data/smi-data.db
Done.


name
okcupid
credit_ger
housing_ger
rki_covid19
telco_churn


### Arbeit mit dem OkCupid-Datensatz
Nun können wir die bekannten Befehle nutzen, um Daten abzufragen und zu manipulieren.  
In diesem Beispiel siehst du, wie ein Datensatz der Dating-Plattform OKCupid nach Profilen durchsucht wird, die bestimmten Suchkriterien entsprechen:

In [18]:
%%sql 

SELECT *
FROM okcupid
WHERE (drugs="never") AND (income > 40000) AND (age > 20) AND (age < 30)
  AND (sex="f") AND (orientation="straight") AND (pets="likes dogs")
LIMIT 20;

 * sqlite:///data/smi-data.db
Done.


index,username,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
168,user_0000035427,26,fit,mostly anything,socially,never,graduated from college/university,white,68.0,70000,science / tech / engineering,2012-06-18-21-51,"san francisco, california","doesn't have kids, but wants them",straight,likes dogs,agnosticism and laughing about it,f,sagittarius but it doesn't matter,no,english,single
1282,user_0000036454,25,athletic,strictly anything,socially,never,working on college/university,white,68.0,1000000,artistic / musical / writer,2012-06-29-22-39,"san francisco, california","doesn't have kids, but wants them",straight,likes dogs,other and laughing about it,f,sagittarius and it's fun to think about,no,english (fluently),single
8183,user_0000042856,26,fit,,socially,never,graduated from college/university,white,64.0,50000,executive / management,2012-06-26-23-12,"san francisco, california",,straight,likes dogs,,f,leo,no,english,single
10182,user_0000044711,23,average,mostly vegetarian,socially,never,graduated from college/university,"asian, pacific islander",59.0,60000,science / tech / engineering,2012-06-29-08-53,"san francisco, california",doesn't want kids,straight,likes dogs,atheism,f,,no,"english, sign language (poorly)",single
11364,user_0000045796,29,average,mostly anything,socially,never,graduated from masters program,"asian, pacific islander, other",62.0,50000,other,2012-06-15-20-54,"vallejo, california","doesn't have kids, but wants them",straight,likes dogs,catholicism but not too serious about it,f,virgo and it's fun to think about,yes,english (fluently),single
21482,user_0000055111,24,thin,,socially,never,graduated from college/university,asian,62.0,70000,law / legal services,2012-06-27-07-53,"san francisco, california",,straight,likes dogs,catholicism but not too serious about it,f,,no,english,single
21577,user_0000055201,28,average,,socially,never,working on masters program,white,68.0,70000,artistic / musical / writer,2012-06-29-02-31,"san francisco, california",,straight,likes dogs,other and somewhat serious about it,f,scorpio,no,"english (fluently), spanish (okay)",single
21691,user_0000006732,28,average,,socially,never,graduated from college/university,white,66.0,60000,sales / marketing / biz dev,2012-06-29-13-31,"san francisco, california",,straight,likes dogs,,f,aries and it's fun to think about,no,english,single
23141,user_0000056653,26,curvy,mostly vegetarian,socially,never,graduated from college/university,white,67.0,60000,transportation,2012-06-28-07-05,"san francisco, california","doesn't have kids, but might want them",straight,likes dogs,atheism and very serious about it,f,virgo but it doesn't matter,no,"english (fluently), french (poorly), italian (poorly), indonesian (poorly)",single
26342,user_0000059604,26,athletic,,socially,never,graduated from masters program,white,70.0,60000,banking / financial / real estate,2012-06-26-18-58,"san francisco, california",,straight,likes dogs,atheism,f,libra but it doesn't matter,no,"english, spanish (poorly)",single


---
### <span style="color:#FF5D02;">Aufgabe 4: Datenabruf mit SQL</span>
Verändere die Abfrage, sodass sie einen anderen Ausschnitt der Daten abbildet.  
WICHTIG: Behalte den ```LIMIT 20``` Befehl am Ende bei, sonst könnte dein Notebook abstürzen wenn zehntausende Datensätze anzuzeigen wären.

## 5. Verbindung zu APIs herstellen<a id="kapitel5"/>

Für eine REST-API benötigen wir kein SQL - hier werden ggf. Filter in der URL (den "Link") mit angegeben. Wie das jeweils funktioniert, ist der Dokumentation der jeweiligen API zu entnehmen.

Starten wir mit einem einfachen Beispiel: 
>[api.open-notify.org/astros.json](http://api.open-notify.org/astros.json) bietet eine Liste aller derzeit im Weltall lebenden Menschen.   
>Klicke zuerst im Browser auf den Link und sieh dir die Daten an. Dieses Format nennt sich JSON (JavaScript Object Notation, mehr dazu [hier](https://www.w3schools.com/js/js_json_syntax.asp)).  
>Führe im Anschluss den nachfolgenden Code aus, um die Daten mit Python einzulesen.

In [ ]:
import pandas as pd  # Python Paket, das wir auch im weiteren für Datenmanagement verwenden
import requests      # Python Paket, das HTTP-Anfragen (engl. requests) stellen kann

link = "http://api.open-notify.org/astros.json"

# API abfragen
response = requests.get(link)
status = response.status_code

if (status == 200):  # HTTP Statuscode 200 bedeutet "Übertragung erfolgreich"
    print("Abruf erfolgreich!")
else:
    print("Fehler beim Abruf!")

In [ ]:
# Sehen wir uns nun die empfangenen Rohdaten an...
# Welche Struktur erkennst du hier wieder? Wo sind Listen? Wo Dictionaries?
response.json()

In [ ]:
# Nun übernehmen wir die Struktur im Feld "people" (record_path) und erhalten eine Datentabelle
pd.json_normalize(response.json(), record_path="people")

## 6. Übungen Datenbank- und API-Abfragen <a id="kapitel6"/>

### <span style="color:#FF5D02;">Aufgabe 6.1: Datenabruf mit SQL</span>

Die Tabelle ```housing_ger``` in der Datenbank ```smi``` auf dem oben verwendeten Datenbankserver enthält ~190MB Daten von zu vermietenden Wohnungen in Deutschland (Stand April 2020). 

Stelle eine Verbindung zum Datenbankserver her und verwende ```SELECT``` Abfragen, um zu ermitteln, 
- was Neubauwohnungen (obj_newlyConst) in Berlin (obj_regio1) mit 50-55qm (obj_livingSpace) in etwa kosten (obj_totalRent) und
- wieviele Wohnungen aus dem 19. Jahrhundert (obj_yearConstructed) im Landkreis München (geo_krs) angeboten werden.
- Wieviele davon haben Zentralheizung (obj_heatingType)?

### <span style="color:#FF5D02;">Aufgabe 6.2: Datenabruf von APIs</span>
##### 6.2.1. Die API https://anapioficeandfire.com/api gibt unter dem Endpunkt ```/books``` eine Liste aller Game of Thrones Bücher zurück, unter dem Endpunkt ```/characters/<characterId>``` das Profil eines Charakters. Rufe die Buchliste ab sowie das Profil des Charakters mit der Id 583. Wer ist es?
##### 6.2.2. Die API https://corona-api.com gibt unter dem Endpunkt ```/timeline``` die Entwicklung der globalen COVID19-Fälle zurück. Wann wurden die Daten zuletzt aktualisiert?
